In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from sklearn import preprocessing
from tqdm import tqdm
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.utils import np_utils
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import tensorflow as tf

In [2]:
df = pd.read_csv('Processed_Data.csv', engine = 'python')
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...
...,...,...,...,...,...,...
140253,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.5,Positive,congratulation winner participant olympics win...
140254,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.5,Negative,hope wrong see tweet government
140255,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.0,Neutral,tokyo pass baton paris strangest ever olympic ...
140256,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.0,Neutral,paris plan deliver inclusive youth centre gend...


In [3]:
df = df.dropna()
df.shape

(139707, 6)

In [4]:
def ratio(x):
  if x=='Positive':
    return 1
  elif x=='Negative':
    return 2
  else:
    return 0;

In [5]:
df['label'] = df['class'].apply(ratio)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets,label
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...,1
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...,2
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...,0
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...,1
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...,1


In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
df['processed_tweets'] = df['processed_tweets'].astype(str)
x = df['processed_tweets']
y = df['label']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(x)

sequences = tokenizer.texts_to_sequences(x)
data = pad_sequences(sequences, maxlen=50)

In [8]:
data.shape

(139707, 50)

In [9]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test = tts(data,y,test_size = 0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(111765, 50)
(27942, 50)
(111765,)
(27942,)


In [10]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

42087


In [11]:
embedding_dim = 100
print("Building Model")

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length = 50))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Building Model
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           4208700   
_________________________________________________________________
gru (GRU)                    (None, 32)                12768     
_________________________________________________________________
dense (Dense)                (None, 3)                 99        
Total params: 4,221,567
Trainable params: 4,221,567
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(x_train, y_train, validation_split=0.2, epochs=10)

Epoch 1/10
2795/2795 [==============================] - 306s 108ms/step - loss: 0.3992 - accuracy: 0.8575 - val_loss: 0.3091 - val_accuracy: 0.8974
Epoch 2/10
2795/2795 [==============================] - 302s 108ms/step - loss: 0.2570 - accuracy: 0.9127 - val_loss: 0.2930 - val_accuracy: 0.9031
Epoch 3/10
2795/2795 [==============================] - 303s 108ms/step - loss: 0.2040 - accuracy: 0.9309 - val_loss: 0.2975 - val_accuracy: 0.9018
Epoch 4/10
2795/2795 [==============================] - 302s 108ms/step - loss: 0.1646 - accuracy: 0.9448 - val_loss: 0.3204 - val_accuracy: 0.9001
Epoch 5/10
2795/2795 [==============================] - 302s 108ms/step - loss: 0.1333 - accuracy: 0.9550 - val_loss: 0.3447 - val_accuracy: 0.8990
Epoch 6/10
2795/2795 [==============================] - 300s 108ms/step - loss: 0.1112 - accuracy: 0.9627 - val_loss: 0.3812 - val_accuracy: 0.8970
Epoch 7/10
2795/2795 [==============================] - 301s 108ms/step - loss: 0.0951 - accuracy: 0.9675 - val_

In [13]:
model.evaluate(x_test, y_test)

874/874 [==============================] - 7s 8ms/step - loss: 0.5301 - accuracy: 0.8905


[0.5301487445831299, 0.8904516696929932]